In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

PROJECT_JSON = kraft.read_json("../project.json")

PATH = make_path_dict(PROJECT_JSON)

In [ ]:
w = pd.read_csv(PATH["w.tsv"], sep="\t", index_col=0)

w.columns.name = "Factor"

h = pd.read_csv(PATH["h.tsv"], sep="\t", index_col=0)

h.columns.name = PROJECT_JSON["sample_alias"]

In [ ]:
for w_or_h, dataframe, normalization_axis in (("w", w, 1), ("h", h, 0)):

    kraft.plot_heat_map(
        kraft.normalize_dataframe(
            dataframe.iloc[
                kraft.cluster_matrix(dataframe.values, 0),
                kraft.cluster_matrix(dataframe.values, 1),
            ],
            normalization_axis,
            "-0-",
        ),
        layout={"title": {"text": "MF {}".format(w_or_h.title())}},
        html_file_path=os.path.join(
            PATH["{}/".format(w_or_h)], "{}.html".format(w_or_h)
        ),
    )

In [ ]:
for element_type, w_or_h, size_normalization_axis in (
    ("feature", "w", 1),
    ("sample", "h", 0),
):

    if element_type == "feature":

        dataframe = w.loc[
            [
                feature
                for feature in w.index
                if any(
                    feature_to_peek in feature
                    for feature_to_peek in PROJECT_JSON[
                        "{}s_to_peek".format(element_type)
                    ]
                )
            ]
        ]

    elif element_type == "sample":

        dataframe = h[
            [
                sample
                for sample in h.columns
                if any(
                    sample_to_peek in sample
                    for sample_to_peek in PROJECT_JSON[
                        "{}s_to_peek".format(element_type)
                    ]
                )
            ]
        ]

    if dataframe.empty:

        continue

    kraft.plot_bubble_map(
        kraft.normalize_dataframe(dataframe, size_normalization_axis, "-0-"),
        dataframe_color=dataframe,
        layout={
            "title": {
                "text": "{} to Peek".format(
                    PROJECT_JSON["{}_alias".format(element_type)]
                )
            }
        },
        html_file_path=os.path.join(
            PATH["{}/".format(w_or_h)], "{}.peek.html".format(w_or_h)
        ),
    )

In [ ]:
signal = pd.read_csv(PATH["feature_x_sample.signal.tsv"], sep="\t", index_col=0)

signal.columns.name = PROJECT_JSON["sample_alias"]

signal.fillna(value=0, inplace=True)

In [ ]:
feature_x_sample_processed = pd.read_csv(
    PATH["feature_x_sample.processed.tsv"], sep="\t", index_col=0
)

feature_x_sample_processed.columns.name = PROJECT_JSON["sample_alias"]

In [ ]:
for w_or_h, targets, data_signal, data_raw in (
    ("w", w.T, signal.T, feature_x_sample_processed.T),
    ("h", h, signal, feature_x_sample_processed),
):

    kraft.make_match_panels(
        targets,
        {
            "Signal": {"dataframe": data_signal, "data_type": "continuous"},
            "Feature-x-Sample": {"dataframe": data_raw, "data_type": "continuous"},
        },
        n_required_for_match_function=PROJECT_JSON["n_required_for_match_function"],
        n_sampling=0,
        n_permutation=0,
        plot_std=PROJECT_JSON["plot_std"],
    )